<a href="https://colab.research.google.com/github/prav1807/vrpAntColonyOptimisation/blob/initialAntColony/antColonyOptimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import random

In [2]:
# Problem setup
class VRP:
    def __init__(self, num_customers, vehicle_capacity, depot, demand, distance_matrix):
        self.num_customers = num_customers
        self.vehicle_capacity = vehicle_capacity
        self.depot = depot
        self.demand = demand
        self.distance_matrix = distance_matrix

# function to calculate total distance of a route
def calculate_route_distance(route, distance_matrix):
    total_distance = 0
    for i in range(1, len(route)):
        total_distance += distance_matrix[route[i-1]][route[i]]
    total_distance += distance_matrix[route[-1]][route[0]]  # return to depot
    return total_distance

# Initialize pheromones
def initialize_pheromones(num_customers, initial_pheromone_value):
    pheromones = np.full((num_customers + 1, num_customers + 1), initial_pheromone_value)
    return pheromones

# Ant Colony Optimization algorithm for Vehicle Routing Problem
class ACO:
    def __init__(self, vrp, num_ants, alpha, beta, evaporation_rate, Q, num_iterations):
        self.vrp = vrp
        self.num_ants = num_ants
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.Q = Q
        self.num_iterations = num_iterations
        self.pheromones = initialize_pheromones(vrp.num_customers, 1.0)  # Initial pheromone value set to 1.0

    # Ant solution construction
    def construct_solution(self):
        ants_solutions = []
        for ant in range(self.num_ants):
            solution = [self.vrp.depot]  # start from depot
            current_capacity = 0
            visited = set([self.vrp.depot])

            while len(visited) < self.vrp.num_customers + 1:
                current_node = solution[-1]
                next_node = self.select_next_node(current_node, visited)
                if self.vrp.demand[next_node] + current_capacity <= self.vrp.vehicle_capacity:
                    solution.append(next_node)
                    current_capacity += self.vrp.demand[next_node]
                    visited.add(next_node)
                else:
                    solution.append(self.vrp.depot)  # return to depot when capacity is exceeded
                    current_capacity = 0

            solution.append(self.vrp.depot)  # return to depot after visiting all customers
            ants_solutions.append(solution)

        return ants_solutions

    # Probability calculation for ant to select the next node
    def select_next_node(self, current_node, visited):
        probabilities = []
        pheromones = self.pheromones[current_node]
        for next_node in range(len(self.vrp.demand)):
            if next_node not in visited:
                pheromone_value = pheromones[next_node] ** self.alpha
                heuristic_value = (1.0 / self.vrp.distance_matrix[current_node][next_node]) ** self.beta
                probabilities.append(pheromone_value * heuristic_value)
            else:
                probabilities.append(0)  # No probability for already visited nodes

        probabilities = np.array(probabilities)
        probabilities /= probabilities.sum()  # Normalize probabilities
        return np.random.choice(range(len(self.vrp.demand)), p=probabilities)

    # Update pheromones
    def update_pheromones(self, ants_solutions):
        # Evaporate pheromones
        self.pheromones *= (1 - self.evaporation_rate)

        # Deposit pheromones based on the quality of solutions
        for solution in ants_solutions:
            route_distance = calculate_route_distance(solution, self.vrp.distance_matrix)
            pheromone_deposit = self.Q / route_distance
            for i in range(1, len(solution)):
                self.pheromones[solution[i-1]][solution[i]] += pheromone_deposit
                self.pheromones[solution[i]][solution[i-1]] += pheromone_deposit

    # Main ACO loop
    def run(self):
        best_solution = None
        best_distance = float('inf')

        for iteration in range(self.num_iterations):
            # Step 1: Construct solutions
            ants_solutions = self.construct_solution()

            # Step 2: Evaluate and update the best solution
            for solution in ants_solutions:
                distance = calculate_route_distance(solution, self.vrp.distance_matrix)
                if distance < best_distance:
                    best_solution = solution
                    best_distance = distance

            # Step 3: Update pheromones
            self.update_pheromones(ants_solutions)

            print(f"Iteration {iteration + 1}/{self.num_iterations}, Best distance: {best_distance}")

        return best_solution, best_distance


In [4]:
# Testing the basic Ant Colony Optimisation algorithm for VRP ...

# Small VRP instance with 5 customers + depot
num_customers = 5
vehicle_capacity = 15
depot = 0  # Depot is at index 0

# Customer demands
demand = [0, 3, 5, 4, 7, 6]  # Depot demand is 0

# Distance matrix (6x6: depot + 5 customers)
distance_matrix = np.array([
    [0, 2, 9, 10, 5, 6],  # Distances from depot
    [2, 0, 4, 6, 8, 3],   # Distances from customer 1
    [9, 4, 0, 3, 7, 8],   # Distances from customer 2
    [10, 6, 3, 0, 9, 4],  # Distances from customer 3
    [5, 8, 7, 9, 0, 2],   # Distances from customer 4
    [6, 3, 8, 4, 2, 0]    # Distances from customer 5
])

# Define the VRP problem
vrp_problem = VRP(
    num_customers=num_customers,
    vehicle_capacity=vehicle_capacity,
    depot=depot,
    demand=demand,
    distance_matrix=distance_matrix
)

# ACO parameters
num_ants = 10
alpha = 1.0  # Influence of pheromone
beta = 2.0   # Influence of heuristic (distance)
evaporation_rate = 0.5  # Pheromone evaporation rate
Q = 100  # Constant for pheromone deposit
num_iterations = 100  # Number of iterations

# Initialize and run ACO
aco = ACO(
    vrp=vrp_problem,
    num_ants=num_ants,
    alpha=alpha,
    beta=beta,
    evaporation_rate=evaporation_rate,
    Q=Q,
    num_iterations=num_iterations
)

best_solution, best_distance = aco.run()

# Output the best solution found
print(f"\nBest solution (route): {best_solution}")
print(f"Best total distance: {best_distance}")


Iteration 1/100, Best distance: 32
Iteration 2/100, Best distance: 32
Iteration 3/100, Best distance: 32
Iteration 4/100, Best distance: 32
Iteration 5/100, Best distance: 32
Iteration 6/100, Best distance: 32
Iteration 7/100, Best distance: 32
Iteration 8/100, Best distance: 32
Iteration 9/100, Best distance: 32
Iteration 10/100, Best distance: 32
Iteration 11/100, Best distance: 32
Iteration 12/100, Best distance: 32
Iteration 13/100, Best distance: 32
Iteration 14/100, Best distance: 32
Iteration 15/100, Best distance: 32
Iteration 16/100, Best distance: 32
Iteration 17/100, Best distance: 32
Iteration 18/100, Best distance: 32
Iteration 19/100, Best distance: 32
Iteration 20/100, Best distance: 32
Iteration 21/100, Best distance: 32
Iteration 22/100, Best distance: 32
Iteration 23/100, Best distance: 32
Iteration 24/100, Best distance: 32
Iteration 25/100, Best distance: 32
Iteration 26/100, Best distance: 32
Iteration 27/100, Best distance: 32
Iteration 28/100, Best distance: 32
I